In [2]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import OLSInfluence
from statsmodels.stats.anova import anova_lm
import statsmodels.api as sm
from statsmodels.graphics.gofplots import ProbPlot
from statsmodels.stats.diagnostic import het_breuschpagan, het_goldfeldquandt
from statsmodels.stats.stattools import durbin_watson



In [ ]:
# Leer los datos del archivo IMCinfantil.xlsx
IMCinfantil = pd.read_excel(r"C:/Users/cecim/Documents/Ce/AID/DataSets/IMCinfantil.xlsx")

# Visualizar las primeras filas del dataframe
IMCinfantil.head()

# Realizar el diagrama de dispersión de las variables PESO y CC
plt.scatter(IMCinfantil['PESO'], IMCinfantil['CC'], c='red', marker='o')
plt.xlabel("Peso")
plt.ylabel("Circunferencia de la Cintura")
plt.title("Peso vs Circunferencia de la Cintura")
plt.show()

# Calcular la correlación de Pearson entre PESO y CC
correlation_pearson = IMCinfantil['PESO'].corr(IMCinfantil['CC'])
print("Correlación de Pearson entre PESO y CC:", correlation_pearson)



In [ ]:
# Realizar el diagrama de dispersión de las variables Cabeza y Alas del dataset gorriones.xlsx
gorriones = pd.read_excel("C:/Users/cecim/Documents/Ce/AID/DataSets/gorriones.xlsx")

# Visualizar las primeras filas del dataframe
gorriones.head()

# Realizar el diagrama de dispersión de las variables Cabeza y Alas
plt.scatter(gorriones['Cabeza'], gorriones['Alas'], c='red', marker='o')
plt.xlabel("Cabeza")
plt.ylabel("Alas")
plt.title("Cabeza vs Alas")
plt.show()

# Calcular la correlación de Pearson entre Cabeza y Alas
correlation_pearson_gorriones = gorriones['Cabeza'].corr(gorriones['Alas'])
print("Correlación de Pearson entre Cabeza y Alas:", correlation_pearson_gorriones)

# Crear un modelo de regresión lineal simple para predecir Largo a partir de Alas en gorriones
model_gorriones = ols('Largo ~ Alas', data=gorriones).fit()

# Obtener el resumen del modelo
print(model_gorriones.summary())

# Realizar el gráfico de la recta de regresión
sns.scatterplot(x='Alas', y='Largo', data=gorriones, color='red', marker='o')
plt.title("Largo vs Alas")
sns.regplot(x='Alas', y='Largo', data=gorriones, color='black', scatter=False)
plt.show()

# Generar nuevos datos en el rango de Alas
nuevosdatos = np.linspace(min(gorriones['Alas']), max(gorriones['Alas']), 100)



In [ ]:
# Predecir Largo para los nuevos datos de Alas
predicciones = model_gorriones.get_prediction(pd.DataFrame({'Alas': nuevosdatos})).summary_frame()

# Obtener los intervalos de confianza
intervalos_confianza = predicciones[['mean', 'mean_ci_lower', 'mean_ci_upper']]

# Visualizar los intervalos de confianza
print(intervalos_confianza.head())



In [ ]:
# Realizar el gráfico con las curvas de puntos de extremos superiores e inferiores de los intervalos
sns.scatterplot(x='Alas', y='Largo', data=gorriones, color='red', marker='o')
plt.title("Largo vs Alas")
sns.lineplot(x=nuevosdatos, y=intervalos_confianza['mean'], color='black')
plt.fill_between(nuevosdatos, intervalos_confianza['mean_ci_lower'], intervalos_confianza['mean_ci_upper'], color='blue', alpha=0.3)
plt.show()

# Realizar el histograma de los residuos
residuos = model_gorriones.resid
plt.hist(residuos, bins=10, density=True, color='blue', edgecolor='black')
plt.title("Histograma de los Residuos")
plt.show()

# Realizar el gráfico cuantil-cuantil (Q-Q) de los residuos
qq_plot = ProbPlot(residuos)
qq_plot.qqplot(line='45', alpha=0.7, color='blue', lw=1)
plt.title("Gráfico Q-Q de los Residuos")
plt.show()

# Realizar el test de Shapiro-Wilk para la normalidad de los residuos
shapiro_test = stats.shapiro(residuos)
print("Resultado del Test de Shapiro-Wilk para los Residuos:")
print("Estadístico de Prueba:", shapiro_test[0])
print("Valor p:", shapiro_test[1])

# Realizar el gráfico de análisis de homocedasticidad
influence = OLSInfluence(model_gorriones)
sns.scatterplot(x=model_gorriones.fittedvalues, y=residuos, color='blue', marker='o')
plt.axhline(y=0, color='black', linestyle='--', lw=1)
plt.title("Análisis de Homocedasticidad")
plt.xlabel("Valores Ajustados")
plt.ylabel("Residuos")
plt.show()

# Realizar el test de Breusch-Pagan para la homocedasticidad
bp_test = het_breuschpagan(residuos, model_gorriones.model.exog)
print("Resultado del Test de Breusch-Pagan para Homocedasticidad:")
print("Estadístico de Prueba:", bp_test[0])
print("Valor p:", bp_test[1])

# Realizar el gráfico de análisis de independencia de las observaciones
sns.scatterplot(x=range(len(residuos)), y=residuos, color='blue', marker='o')
plt.axhline(y=0, color='black', linestyle='--', lw=1)
plt.title("Análisis de Independencia de las Observaciones")
plt.xlabel("Índice de Observación")
plt.ylabel("Residuos")
plt.show()

# Realizar el test de Durbin-Watson para la autocorrelación de los residuos
dw_test = durbin_watson(residuos)
print("Resultado del Test de Durbin-Watson:")
print("Estadístico de Durbin-Watson:", dw_test)

# Realizar el test de Henze-Zirkler para evaluar la normalidad multivariada
gorriones['Largo_pred'] = model_gorriones.fittedvalues
gorriones['Residuos'] = residuos
cabeza_alas = gorriones[['Cabeza', 'Alas']]

# Usar el Test Henze-Zirkler para evaluar normalidad multivariada (bivariada en este caso)
multivariate_normality = mvnormtest.mvn(data=cabeza_alas, mvnTest="hz")
print("Resultado del Test de Henze-Zirkler para Normalidad Multivariada:")
print(multivariate_normality)
print("Resultado de la Prueba de Normalidad Multivariada:", multivariate_normality.mvnp)
